In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report

# ==========================================
# 1️⃣ Load and preprocess CIFAR-10 dataset
# ==========================================
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train.astype('float32') / 255.0, x_test.astype('float32') / 255.0

# One-hot encode labels
y_train_cat, y_test_cat = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Class names
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

print("Dataset loaded successfully!")
print(f"Train shape: {x_train.shape}, Test shape: {x_test.shape}")

# ==========================================
# 2️⃣ Load Pretrained VGG16 Base Model
# ==========================================
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze all convolutional layers (❄️ frozen weights)
for layer in base_model.layers:
    layer.trainable = False

print("\nVGG16 base model loaded and frozen.")
base_model.summary()

# ==========================================
# 3️⃣ Build Full Model (Feature Extractor + Classifier)
# ==========================================
model = Sequential([
    base_model,
    Flatten(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\nFull model architecture:")
model.summary()

# ==========================================
# 4️⃣ Define Callbacks
# ==========================================
callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    )
]

# ==========================================
# 5️⃣ Train the Model
# ==========================================
print("\nStarting training with frozen base layers...")
history = model.fit(
    x_train, y_train_cat,
    validation_data=(x_test, y_test_cat),
    batch_size=64,
    epochs=30,
    callbacks=callbacks,
    verbose=1
)

# ==========================================
# 6️⃣ Evaluate Model
# ==========================================
print("\nEvaluating model on test set...")
test_loss, test_accuracy = model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# ==========================================
# 7️⃣ Plot Training History
# ==========================================
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# ==========================================
# 8️⃣ Predictions and Visualization
# ==========================================
def show_predictions(model, x_test, y_test, classes, num_images=8):
    predictions = model.predict(x_test)
    predicted_classes = np.argmax(predictions, axis=1)

    indices = np.random.choice(len(x_test), num_images, replace=False)
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))

    for i, idx in enumerate(indices):
        row, col = i // 4, i % 4
        ax = axes[row, col]
        img = x_test[idx]
        true_label = classes[y_test[idx][0]]
        pred_label = classes[predicted_classes[idx]]
        confidence = np.max(predictions[idx])
        color = 'green' if y_test[idx][0] == predicted_classes[idx] else 'red'

        ax.imshow(img)
        ax.set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}',
                     color=color, fontsize=9)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

    correct = sum(1 for idx in indices if y_test[idx][0] == np.argmax(predictions[idx]))
    print(f"Accuracy on displayed images: {correct}/{num_images} ({correct/num_images*100:.1f}%)")

print("\nShowing sample predictions...")
show_predictions(model, x_test, y_test, classes)

# ==========================================
# 9️⃣ Confusion Matrix & Classification Report
# ==========================================
print("\nGenerating confusion matrix and detailed report...")
predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print("\nClassification Report:")
print(classification_report(y_test, predicted_classes, target_names=classes))

# ==========================================
# 🔟 Summary Stats
# ==========================================
print("\nModel Summary:")
print(f"Total Parameters: {model.count_params():,}")
print(f"Frozen Layers: {sum(not l.trainable for l in model.layers[0].layers)}")
print(f"Trainable Layers: {sum(l.trainable for l in model.layers[0].layers)}")


Dataset loaded successfully!
Train shape: (50000, 32, 32, 3), Test shape: (10000, 32, 32, 3)
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step

VGG16 base model loaded and frozen.


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

...

 Total params: 15,113,290 (57.65 MB)

 Trainable params: 397,578 (1.52 MB)

 Non-trainable params: 14,715,712 (56.14 MB)


Starting training with frozen base layers...
Epoch 1/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 135s 171ms/step - accuracy: 0.4513 - loss: 1.5669 - val_accuracy: 0.5587 - val_loss: 1.2699 - learning_rate: 5.0000e-04
Epoch 2/30
387/782 ━━━━━━━━━━━━━━━━━━━━ 1:00 153ms/step - accuracy: 0.5271 - loss: 1.3409

KeyboardInterrupt: 